# Final Experiment for Diploma thesis Bitcoin price prediction


#### Load all own need ipynb

In [5]:
%run Imports.ipynb
%run DataFrameLoader.ipynb

In [6]:
df = LoadBaseDataset()

In [7]:
df.head()

,date,noTweets,sum_positive,sum_negative,sum_neutral,sum_compound,avg_sum_positive,avg_sum_negative,avg_sum_neutral,avg_sum_compound,...,amd_price,oil_price,intel_volume,nvidia_price,nvidia_volume,oil_volume,amd_volume,dayOfWeek_sin,dayOfWeek_cos,isWeekend
0,2019-03-27 00:00:00,2652,238.597,91.014,2322.397,521.1693,0.089744,0.034314,0.875566,0.196456,...,25.700001,12.257000,1.965600e+07,177.899994,1.185930e+07,3000.000000,8.858530e+07,0.974928,-0.222521,0
1,2019-03-27 01:00:00,2674,230.083,87.836,2356.083,482.5923,0.086013,0.032536,0.881077,0.180254,...,25.675001,12.244375,1.883700e+07,177.885411,1.136517e+07,2875.498083,8.489425e+07,0.974928,-0.222521,0
2,2019-03-27 02:00:00,2407,233.193,69.714,2104.068,530.6851,0.096801,0.028666,0.874117,0.220191,...,25.650001,12.231750,1.801800e+07,177.870828,1.087104e+07,2750.996167,8.120319e+07,0.974928,-0.222521,0
3,2019-03-27 03:00:00,2322,230.689,66.212,2025.090,546.9008,0.099053,0.028424,0.872093,0.235142,...,25.625001,12.219125,1.719901e+07,177.856245,1.037691e+07,2626.494250,7.751214e+07,0.974928,-0.222521,0
4,2019-03-27 04:00:00,2207,213.122,61.475,1932.409,521.1172,0.096511,0.027639,0.875396,0.236067,...,25.600001,12.206500,1.638001e+07,177.841662,9.882780e+06,2501.992333,7.382109e+07,0.974928,-0.222521,0
